In [5]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import pickle

## LSTM으로 네이버 영화 리뷰 감성 분류

In [6]:
np.random.seed(5)

In [7]:
X_train = np.load("./save/X_train.npy")
X_test = np.load("./save/X_test.npy")
y_train = np.load("./save/y_train.npy")
y_test = np.load("./save/y_test.npy")
with open('./save/vocab_size.p', 'rb') as file:
    vocab_size = pickle.load(file)

In [8]:
model_name = 'model_team3.h5'

T = X_train.shape[1]

i = Input(shape=(T,))
x = Embedding(vocab_size, 128)(i)
x = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = LSTM(32, return_sequences=True)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dense(32, activation='sigmoid')(x)
x = Flatten()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 128)           997376    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 128)           65664     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15, 128)           65664     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 7, 32)             20608 

In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(model_name, monitor='val_acc', mode='max', 
                     verbose=1, save_best_only=True)

In [10]:
model.compile(optimizer=Adam(lr=0.002,
                             beta_1=0.9,
                             beta_2=0.999, 
                             epsilon=None,
                             decay=0.01,
                             amsgrad=False),
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=15, 
                    callbacks=[es, mc], batch_size=60, validation_split=0.1)

Train on 21561 samples, validate on 2396 samples
Epoch 1/15
21561/21561 [==============================] - 14s 666us/sample - loss: 0.4139 - accuracy: 0.8197 - val_loss: 0.3149 - val_accuracy: 0.8694
Epoch 2/15
21561/21561 [==============================] - 13s 623us/sample - loss: 0.2454 - accuracy: 0.9033 - val_loss: 0.3063 - val_accuracy: 0.8744
Epoch 3/15
21561/21561 [==============================] - 13s 616us/sample - loss: 0.1781 - accuracy: 0.9367 - val_loss: 0.3150 - val_accuracy: 0.8710
Epoch 4/15
21561/21561 [==============================] - 14s 630us/sample - loss: 0.1315 - accuracy: 0.9564 - val_loss: 0.3702 - val_accuracy: 0.8681
Epoch 5/15
21561/21561 [==============================] - 14s 640us/sample - loss: 0.0998 - accuracy: 0.9710 - val_loss: 0.3992 - val_accuracy: 0.8614
Epoch 6/15
21561/21561 [==============================] - 14s 636us/sample - loss: 0.0785 - accuracy: 0.9782 - val_loss: 0.4390 - val_accuracy: 0.8635
Epoch 00006: early stopping


In [11]:
from tensorflow.keras.models import load_model
model.save('./save/'+model_name)

In [12]:
loaded_model = load_model('./save/'+model_name)
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

12594/12594 [==============================] - 3s 229us/sample - loss: 0.3300 - accuracy: 0.8980

 테스트 정확도: 0.8980
